In [5]:
import keras
import numpy as np
import tensorflow as tf

'''from keras.models import Sequential
from keras.layers import *
from keras import backend as k
from keras import optimizers
from keras.models import Model
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import load_model
from keras.engine.topology import Layer'''
from keras.datasets import mnist

In [6]:
batchsize = 64
n_z = 20
layers = tf.contrib.layers

In [17]:
def encoder(inputs):

    encode = layers.conv2d(inputs, 32, (2, 2), stride=2, activation_fn = tf.nn.relu)

    encode = layers.max_pool2d(encode,(2, 2))

    encode = layers.conv2d(encode, 32, (2, 2), stride=2, activation_fn = tf.nn.relu)
    
    encode = layers.max_pool2d(encode, (2, 2))

    encode = layers.flatten(encode)

    encode = tf.layers.dense(encode, 512, activation = tf.nn.relu)
    
    encode = layers.dropout(encode, 0.5)

    mu = tf.layers.dense(encode, n_z)

    log_sigma = tf.layers.dense(encode, n_z)
    
    return mu, log_sigma

In [23]:
def decoder(latent_vector): 
    
    decode = tf.layers.dense(latent_vector, 512, activation = tf.nn.relu)
    
    decode = tf.reshape(decode, (14, 14, 1))
    
    decode = layers.conv2d(decode, 32, (2, 2), stride=2, activation_fn = tf.nn.relu)
    
    # Tensorflow?
    decode = tf.keras.layers.UpSampling2D((2, 2))(decode)
    
    decode = layers.conv2d(decode, 32, (2, 2), stride=2, activation_fn = tf.nn.relu)
    
    # Tensorflow?
    decode =  tf.keras.layers.UpSampling2D((2, 2))(decode)
    
    out = layers.conv2d(decode, 1, (2, 2), activation_fn = tf.sigmoid)
    
    mu_out = tf.layers.dense(out, n_z)

    log_sigma_out = tf.layers.dense(out, n_z)
    
    return mu_out, log_sigma_out, out

In [ ]:
def loss():
    # p_z = gaussian(0,1)
    # q_z_x = gaussian(mean_y, e^ln_var_y) - check this exp
    # p_x_z = gaissian(mean_x, e^ln_var_x)
    # L = F.gaussian_kl_dvergance(q_z_x, p_z) - F.gaussian_nll(p_x_z)
        
    #? What about input x?
        
    # Why use log variance instead of variance:

In [19]:
def sample_z(mu, log_sigma):
    eps = tf.random_normal(shape=(batchsize, n_z), mean=0., stddev=1.)
    return mu + tf.exp(log_sigma / 2) * eps

In [9]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
batches = int(len(x_train)/batchsize)

In [ ]:
type(x_train)

In [ ]:
x_train.shape

In [10]:
def batch_generator():
    
    while True:

        for batch in range(batches):
            x = []
            
            data = x_train[batch * batchsize: (batch + 1) * batchsize]
            for i in data:
                x.append(np.reshape(i, (28, 28, 1)))
            
            x_train1 = np.asarray(x)
            yield (x_train1, x_train1) # the data is also the target (since we're re-generating images)

In [24]:
inputs1 = tf.keras.Input(shape = (28, 28, 1))

mu, log_sigma = encoder(inputs1)

latent_vector = sample_z(mu, log_sigma)

mu_out, log_sigma_out, out_img = decoder(latent_vector)

model = Model(inputs = input1, outputs = out_img)

ValueError: Cannot reshape a tensor with 32768 elements to shape [14,14,1] (196 elements) for 'Reshape' (op: 'Reshape') with input shapes: [64,512], [3] and with input tensors computed as partial shapes: input[1] = [14,14,1].

In [ ]:
model.fit_generator(batch_generator(), steps_per_epoch = batches, epochs=10, shuffle = True, callbacks = callback_list)